In [1]:
from efficient_rerank import run_pipeline, XLMCometEmbeds
import torch
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
import pickle
import pandas as pd
import os
from src.recom_search.model.beam_node_reverse import ReverseNode
import numpy as np
import math 
from new_flatten_lattice import get_dictlist
import flatten_lattice as fl
import json
import ast
import random
xlm_tok = fl.bert_tok


2022-11-26 05:56:23.823065: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-26 05:56:23.823087: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# let's use this as our evaluation metric for diversity
def get_unique_ngrams(sentence, tok, n, uns):
    toks = tok(sentence).input_ids
    #print(toks)
    for i in range(len(toks)-n):
        tmp = ""
        for j in range(i, i+n):
            tmp = tmp+"_"+str(toks[j])
        uns.add(tmp)

def cand_unique_ngrams(sentences, tok, n):
    uniques = set()
    for s in sentences:
        get_unique_ngrams(s, tok, n, uniques)
    return uniques

In [4]:
# TODO something that gets metrics for diverse decoding top k

# load in model, for french
#del model
model = XLMCometEmbeds(drop_rate=0.1)
model.load_state_dict(torch.load("./torchsaved/maskedcont4.pt"))
model.eval()
torch.cuda.memory_allocated(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0

In [5]:
base = "frtest_reversed/"

In [6]:
# get output for a single index out of available graphs
def test_graph_ind(ind, basedir, scofunct):
    g = pickle.load(open(basedir+str(ind), 'rb'))
    #if g['input'] in old['src']:
    #    return None, None, None
    #try:
    global usedlist
    usedlist = []
    options = []
    # TODO add a verbose option for efficient reranking so that it doesn't blow up nb
    return g['input'], g['ref'], run_pipeline(g, model, scofunct, False, 10)

# get predictions for a bunch of stuff
def get_all_preds(basedir, scofunct):
    l = len(os.listdir(basedir))
    result = []
    print("will predict total of ", l)
    for i in range(l):
        inp, r, p = test_graph_ind(i, basedir, scofunct)
        result.append({
            'src':inp,
            'hyp':p,
            'ref':r
        })
        print(i)
    result = pd.DataFrame(result)
    return result

In [7]:
WEIGHT=50
DIVWEIGHT = 1
# should this be more complex
def weightadddiverse (node, used):
    global usedlist
    if hasattr(node, "prob"):
        #pcnt+=1
        if node.token_idx in used:
            # TODO maybe we need to add an element that factors in position as well
            return math.log(node.prob) + WEIGHT*node.score - DIVWEIGHT
        return math.log(node.prob) + WEIGHT*node.score
    else:
        #npcnt+=1
        return 1

In [ ]:
apreds = get_all_preds(base, weightadddiverse)

In [3]:
#apreds.to_csv("div1latpredsfr-en.csv")
apreds = pd.read_csv("div1latpredsfr-en.csv")

In [7]:
ast.literal_eval(apreds.loc[2]['hyp'])

['</s> Another variable is the extent to which other developed countries’ monetary policies will become more flexible.',
 '</s> Another variable is the degree to which other developed countries’ monetary policies will ease.',
 '</s> Another variable is the extent to which monetary policies in other developed economies will be flexible.',
 '</s> Another variable is the extent to which the monetary policies of other developed countries become more flexible.',
 '</s> Another variable is the extent to which other developed countries’ monetary policies will ease.',
 '</s> Another variable is the extent to which other developed countries’ monetary policies will ease.',
 '</s> Another variable is the extent to which other developed countries’ monetary policies will ease.',
 '</s> Another variable is the extent to which other developed countries’ monetary policies will ease.',
 '</s> Another variable is the extent to which other developed countries’ monetary policies will ease.',
 '</s> Anothe

In [134]:
LIM = 3
def evaluate_diversity_all (preds, n):
    totdiv = 0 
    ps = 'hyp'
    if 'hyp' not in preds.keys():
        ps = 'ahyp'
    for h in preds[ps]:
        if type(h) is str:
            hyps = ast.literal_eval(h)
        else:
            hyps = h
        if len(hyps)>LIM:
            hyps = hyps[:LIM]
        #print(len(cand_unique_ngrams(row['hyp'], xlm_tok, 3)))
        #print(len(cand_unique_ngrams(hyps, xlm_tok, 3)))
        totdiv+=len(cand_unique_ngrams(hyps, xlm_tok, n))
    return totdiv / len(preds)

In [135]:
evaluate_diversity_all(apreds, 3)

51.38209606986899

In [89]:
# TODO something that gets metrics for beam search top k
beampreds = pd.read_csv("testbeam50fr_en.csv")

In [91]:
bpclump = beampreds.groupby("ref").agg(list)
    

In [136]:
evaluate_diversity_all(bpclump, 3)

41.28384279475983